In [34]:
"""Libraries"""

# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

import numpy as np   # matrices, math
import time   # runtime measurement
from multiprocessing import Pool   # multithreading
import importlib   # reload changes you made

# my own file:
import diffeq as de    # full bubble model
importlib.reload(de)   # reload changes you made

<module 'diffeq' from 'F:\\OneDrive - Budapesti Műszaki és Gazdaságtudományi Egyetem\\TDK\\Letisztázott programok\\diffeq.py'>

In [3]:
"""Control parameter ranges and division"""
# a list for each control parameter, containing all the possible values

ranges = de.dotdict(dict(
    # Equilibrium radius [um --> m]
    R_E = [1e-6*x for x in [0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 10.0, 15.0, 20.0]],
    # R_star / R_E [-]
    ratio = [2.0, 3.0, 5.0, 7.0, 8.0, 10.0, 14.0, 18.0, 25.0, 32.0],
    # Ambient pressure [Pa]
    P_inf = [1e5*x for x in [1.0, 10.0, 20.0, 50.0]],
    # Accommodation coeff. for evaporation [-]
    alfa_M = [0.05],
    # Ambient temperature [°C --> K]
    T_inf = [273.15+x for x in [30.0]],
    # Surface tension modifier [-]
    surfactant = [1.0]
))

# print total combinations:
for key in ranges:
    print(f'{key}: {len(ranges[key])}')
print(f'total combinations: {np.prod([len(ranges[key]) for key in ranges])}')

R_E: 10
ratio: 10
P_inf: 4
alfa_M: 1
T_inf: 1
surfactant: 1
total combinations: 400


In [51]:
"""Get all combinations"""
# Make a list, with one dictionary for eachy parameter combinations

cpars = []
ID = 1
for R_E in ranges.R_E:
    for ratio in ranges.ratio:
        for P_inf in ranges.P_inf:
            for alfa_M in ranges.alfa_M:
                for T_inf in ranges.T_inf:
                    for surfactant in ranges.surfactant:
                        cpar = dict(
                            ID=ID,
                            R_E=R_E, # [m]
                            ratio=ratio, # [-]
                            P_inf=P_inf, # [Pa]
                            alfa_M=alfa_M, # [-]
                            T_inf=T_inf, # [m]
                            surfactant=surfactant, # [-]
                        )
                        cpar['P_v'] = de.VapourPressure(cpar['T_inf']) # [Pa]
                        cpars.append(cpar)
                        ID += 1

In [49]:
"""Parameter study, multithread"""
# Runs each combinations in cpars, and saves the results into CSV files
# use Pool(processes=...) to limit number of threads being used.
# If you want to use your PC while simulations are running, I recommend using processes=n-1 or n-2, if your CPU has n cores
# # use pool.imap(...) instead of pool.imap_unordered(...) to preserve order in which cpars was made

max_lines = 100    # maximum length of a CSV

start = time.time()
file = de.Make_dir('test')
file.new_file()
with Pool() as pool:
    results = pool.imap_unordered(de.simulate, cpars)

    for data in results:
        if file.lines > max_lines:
            file.close()
            file.new_file()
        data = de.dotdict(data)
        file.write_line(data)
        print(f'index: {data.ID}/{len(cpars)},   error_code: {data.error_code},   steps: {data.steps},   runtime: {data.elapsed_time: .2f} [s]   |   R_E={1e6*data.R_E: .2f} [um]; ' + 
              f'ratio={data.ratio: .2f} [-]; P_inf={1e-5*data.P_inf: .2f} [bar]; alfa_M={data.alfa_M: .2f} [-]; T_inf={273.15+data.T_inf: .2f} [°C]; '+
              f'surfactant={100*data.surfactant: .0f} [%]   |   hydrogen production: {data.energy: .1f} [MJ/kg]                                       ', end='\r')
        
file.close()
end = time.time()
elapsed = end - start
print(f'\n\nDONE')
print(f"total time: {elapsed: .2f} [s]   ({(elapsed / len(cpars)): .2f} [s/run])")

Folder already exists with 12 csv in it
index: 398/400,   error_code: 1,   steps: 22637,   runtime:  15.78 [s]   |   R_E= 20.00 [um]; ratio= 32.00 [-]; P_inf= 10.00 [bar]; alfa_M= 0.05 [-]; T_inf= 576.30 [°C]; surfactant= 100 [%]   |   hydrogen production:  217119.7 [MJ/kg]                                                           

DONE
total time:  359.28 [s]   ( 0.90 [s/run])
